### Solution for scanners plus ####

**Initialising the data**

In [1]:
import pandas as pd

In [2]:
iweight = 0.24  # initial allocation weight

df = pd.DataFrame([[1600, 600, iweight, 800000], [8800, 2800, 1 - iweight, 2400000]], columns=["Price", "VarCost", "Weight", "FixCost"], index=["Home", "Pro"])
df = df.assign(CM=df["Price"] - df["VarCost"])
df

,Price,VarCost,Weight,FixCost,CM
Home,1600,600,0.24,800000,1000
Pro,8800,2800,0.76,2400000,6000


In [3]:
oh = 1_000_000

total_home_units = 2400
total_pro_units = 1400

high_home_units = 1400
high_pro_units = 800

In [4]:
low_home_units = total_home_units - high_home_units
low_pro_units = total_pro_units - high_pro_units

df_units = pd.DataFrame([[low_home_units, low_pro_units], [high_home_units, high_pro_units]], index=["Lo", "Hi"], columns=["Home", "Pro"])
df_units

,Home,Pro
Lo,1000,600
Hi,1400,800


**Answers to A and B**
Determine the revenues

In [5]:
df_rev = df_units * df["Price"]
df_rev

,Home,Pro
Lo,1600000,5280000
Hi,2240000,7040000


**Answers to C**
Determine the profits using the uni-rate system

In [6]:
df_c = df_units * df["CM"] - df["FixCost"] - df["Weight"] * oh
df_c

,Home,Pro
Lo,-40000.0,440000.0
Hi,360000.0,1640000.0


In [7]:
# Mean
print(df_c.mean().to_markdown(floatfmt=",.0f"))

|      |         0 |
|:-----|----------:|
| Home |   160,000 |
| Pro  | 1,040,000 |


In [8]:
# Standard deviation
print(df_c.std().to_markdown(floatfmt=",.0f"))

|      |       0 |
|:-----|--------:|
| Home | 282,843 |
| Pro  | 848,528 |


### The scenarios ###

We need to figure out the allocations based on revenues. In order to do this, for each model we need the revenues for each scenario of High and Low revenues

In [9]:
df_scen = pd.merge(df_rev['Home'], df_rev['Pro'], how='cross')
df_scen.index = ['LoLo', 'LoHi', 'HiLo', 'HiHi']
df_scen

,Home,Pro
LoLo,1600000,5280000
LoHi,1600000,7040000
HiLo,2240000,5280000
HiHi,2240000,7040000


Determine the total revenues for each scenario and model

In [10]:
df_scen = df_scen.assign(Total=df_scen.sum(axis=1))
df_scen

,Home,Pro,Total
LoLo,1600000,5280000,6880000
LoHi,1600000,7040000,8640000
HiLo,2240000,5280000,7520000
HiHi,2240000,7040000,9280000


We can now determine the allocation weights by **div**iding the model revenues by the Total of revenues for each model.

In [11]:
df_weights  =  df_scen[['Home', 'Pro']].div(df_scen['Total'], axis=0).round(2)
df_weights

,Home,Pro
LoLo,0.23,0.77
LoHi,0.19,0.81
HiLo,0.30,0.70
HiHi,0.24,0.76


As you can see, if one model sells a lot, then it gets allocated more of the overhead.

Also, if a manager of one model massively slacks, then it attracts less overhead. The overhead will be allocated to ... the other model. Uh oh! This creates an incentive to monitor the manager of the other model!

---

We can now determine the allocation:

In [12]:
df_w_alloc = df_weights * oh
df_w_alloc

,Home,Pro
LoLo,230000.0,770000.0
LoHi,190000.0,810000.0
HiLo,300000.0,700000.0
HiHi,240000.0,760000.0


**Determine the profit before allocation**

Before we allocate, we need to know the profit before allocation.

Multiply the sales units matrix by the contribution margin per unit and deduct fixed costs.

In [13]:
df_gross_profit = df_units * df["CM"] - df["FixCost"]
df_gross_profit

,Home,Pro
Lo,200000,1200000
Hi,600000,2400000


Now let's  get the allocations for the Home model from the allocation matrix above and wrap them up in a 2 x 2 matrix:

In [14]:
df_w_alloc_home = pd.DataFrame(df_w_alloc['Home'].values.reshape(2, 2), columns=["ProLo", "ProHi"], index=["Lo", "Hi"]).T
df_w_alloc_home

,Lo,Hi
ProLo,230000.0,300000.0
ProHi,190000.0,240000.0


Lastly, deduct (**sub**tract) the allocated overhead from the profits:

In [15]:
df_net_home = df_gross_profit['Home'].sub(df_w_alloc_home, axis=0)
df_net_home

,Lo,Hi
ProLo,-30000.0,300000.0
ProHi,10000.0,360000.0


Do the same for the Pro model

In [16]:
df_w_alloc_pro =  pd.DataFrame(df_w_alloc['Pro'].values.reshape(2, 2), columns=["Lo", "Hi"], index=["HomeLo", "HomeHi"])
df_net_pro = df_gross_profit['Pro'].sub(df_w_alloc_pro, axis=0)
df_net_pro

,Lo,Hi
HomeLo,430000.0,1590000.0
HomeHi,500000.0,1640000.0


### Analysis ###

In [17]:
df_net_home.mean()

Lo    -10000.0
Hi    330000.0
dtype: float64

In [18]:
#  Compare to our previous result:
df_c['Home']

Lo    -40000.0
Hi    360000.0
Name: Home, dtype: float64

In [19]:
#  Compare to our previous result - overall expected profits
print(df_net_home.stack().mean(), df_c['Home'].mean())

160000.0 160000.0


In [20]:
#  Compare to our previous result - overall expected profits
print(f"Home:\nStandard deviation after weighted overhead: {df_net_home.stack().std():,.0f}.\nStandard deviation after constant overhead: {df_c['Home'].std():,.0f}.")

Home:
Standard deviation after weighted overhead: 198,494.
Standard deviation after constant overhead: 282,843.


In [21]:
print(f"Pro:\nStandard deviation after weighted overhead: {df_net_pro.stack().std():,.0f}.\nStandard deviation after constant overhead: {df_c['Pro'].std():,.0f}.")

Pro:
Standard deviation after weighted overhead: 664,881.
Standard deviation after constant overhead: 848,528.


---

What are expected profits in case of low units?

In [22]:
print(f"Home, low:\nProfits after weighted overhead: {df_net_home['Lo'].mean():,.0f}.\nSProfts after constant overhead: {df_c['Home'].loc['Lo']:,.0f}.")
print(f"\nPro, low:\nProfits after weighted overhead: {df_net_pro['Lo'].mean():,.0f}.\nProfts after constant overhead: {df_c['Pro'].loc['Lo']:,.0f}.")

Home, low:
Profits after weighted overhead: -10,000.
SProfts after constant overhead: -40,000.

Pro, low:
Profits after weighted overhead: 465,000.
Profts after constant overhead: 440,000.


---

What are expected profits in case of high units:

In [23]:
print(f"Home, high:\nProfits after weighted overhead: {df_net_home['Hi'].mean():,.0f}.\nSProfts after constant overhead: {df_c['Home'].loc['Hi']:,.0f}.")
print(f"\nPro, high:\nProfits after weighted overhead: {df_net_pro['Hi'].mean():,.0f}.\nProfts after constant overhead: {df_c['Pro'].loc['Hi']:,.0f}.")

Home, high:
Profits after weighted overhead: 330,000.
SProfts after constant overhead: 360,000.

Pro, high:
Profits after weighted overhead: 1,615,000.
Profts after constant overhead: 1,640,000.
